<a href="https://colab.research.google.com/github/tanyagupta1/Machine-Learning/blob/main/DL/pytorch/character_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [18]:
with open('/content/anna.txt','r') as f:
  text=f.read()
print(text[0:100])

Chapter 1


Happy families are all alike; every unhappy family is unhappy in its own
way.

Everythin


In [19]:
chars=tuple(set(text))
int_to_char =dict(enumerate(chars))
char_to_int = {ch:i for i,ch in int_to_char.items()}
encode=np.array([char_to_int[c] for c in text])

In [20]:
print(int_to_char)
print(encode[0:100])

{0: 'q', 1: 'T', 2: '!', 3: 'X', 4: '/', 5: 'N', 6: '7', 7: '\n', 8: '6', 9: ';', 10: 'D', 11: 'I', 12: 'R', 13: 'C', 14: 'j', 15: 'G', 16: 'a', 17: ' ', 18: 'V', 19: 'K', 20: 'y', 21: 'W', 22: '*', 23: 'M', 24: '(', 25: "'", 26: '5', 27: 'F', 28: 'z', 29: 'm', 30: '2', 31: 'r', 32: '"', 33: 'l', 34: 'd', 35: ')', 36: '&', 37: 'A', 38: '_', 39: 'J', 40: 'S', 41: 'g', 42: '`', 43: 'b', 44: 'e', 45: 'p', 46: 's', 47: 'Z', 48: '9', 49: '.', 50: '-', 51: ',', 52: 'U', 53: 't', 54: '?', 55: '$', 56: 'n', 57: 'o', 58: 'h', 59: '3', 60: 'x', 61: 'P', 62: '1', 63: 'E', 64: 'w', 65: 'Q', 66: 'Y', 67: 'c', 68: ':', 69: 'H', 70: '0', 71: 'B', 72: 'L', 73: '%', 74: 'O', 75: '4', 76: 'k', 77: '@', 78: 'i', 79: 'v', 80: 'f', 81: '8', 82: 'u'}
[13 58 16 45 53 44 31 17 62  7  7  7 69 16 45 45 20 17 80 16 29 78 33 78
 44 46 17 16 31 44 17 16 33 33 17 16 33 78 76 44  9 17 44 79 44 31 20 17
 82 56 58 16 45 45 20 17 80 16 29 78 33 20 17 78 46 17 82 56 58 16 45 45
 20 17 78 56 17 78 53 46 17 57 64 56  7 64

In [21]:
def one_hot_encode(arr,n_labels):

  one_hot=np.zeros((arr.size,n_labels),dtype=np.float32)
  one_hot[np.arange(one_hot.shape[0]),arr.flatten()] = 1
  one_hot = one_hot.reshape((*arr.shape,n_labels))
  return one_hot

In [22]:
test_seq = np.array([3,5,1])
one_hot = one_hot_encode(test_seq,8)
print(one_hot)

[[0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]]


In [23]:
def get_batches(arr, batch_size, seq_length):
    batch_size_total = seq_length * batch_size
    n_batches = len(arr)//batch_size_total
    
    arr = arr[:n_batches * batch_size_total]
    
    arr = arr.reshape((batch_size,-1))
    print(arr.shape)
    for n in range(0, arr.shape[1], seq_length):
      
        x = arr[:,n:n+seq_length]
        y = np.zeros_like(x)
        try:
          y[:,:-1] , y[:,-1] = x[:,1:] , arr[:,n+seq_length]
        except IndexError:
          y[:,:-1] , y[:,-1] = x[:,1:] , arr[:,0]
        yield x, y

In [24]:
batches = get_batches(encode,8,50)
x,y = next(batches)
print(x[0])
print(y[0])

(8, 248150)
[13 58 16 45 53 44 31 17 62  7  7  7 69 16 45 45 20 17 80 16 29 78 33 78
 44 46 17 16 31 44 17 16 33 33 17 16 33 78 76 44  9 17 44 79 44 31 20 17
 82 56]
[58 16 45 53 44 31 17 62  7  7  7 69 16 45 45 20 17 80 16 29 78 33 78 44
 46 17 16 31 44 17 16 33 33 17 16 33 78 76 44  9 17 44 79 44 31 20 17 82
 56 58]


In [60]:
class CharRNN(nn.Module):
  def __init__(self,tokens,n_hidden=256,n_layers=2,drop_prob=0.5,lr=0.001):
    super().__init__()
    self.drop_prob = drop_prob
    self.n_hidden = n_hidden
    self.n_layers = n_layers
    self.lr = lr
    self.chars = tokens
    self.int_to_char =dict(enumerate(self.chars))
    self.char_to_int={ch:i for i,ch in self.int_to_char.items()}

    self.lstm = nn.LSTM(len(self.chars),n_hidden,n_layers,dropout=drop_prob,batch_first=True)
    self.dropout = nn.Dropout(drop_prob)
    self.fc = nn.Linear(n_hidden,len(self.chars))

  def forward(self,x,hidden):
    r_output,hidden =self.lstm(x,hidden)
    out = self.dropout(r_output)
    out = out.contiguous().view(-1,self.n_hidden)
    out = self.fc(out)
    return out,hidden

  def init_hidden(self, batch_size):

    weight = next(self.parameters()).data

   
    hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
              weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
    
    return hidden

In [54]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
  
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))


In [61]:
net = CharRNN(chars, 512, 2)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [62]:
batch_size = 128
seq_length = 100
n_epochs = 20

# train the model
train(net, encode, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

(128, 13900)
(128, 1500)
Epoch: 1/20... Step: 10... Loss: 3.2600... Val Loss: 3.1989
(128, 1500)
Epoch: 1/20... Step: 20... Loss: 3.1484... Val Loss: 3.1332
(128, 1500)
Epoch: 1/20... Step: 30... Loss: 3.1426... Val Loss: 3.1234
(128, 1500)
Epoch: 1/20... Step: 40... Loss: 3.1116... Val Loss: 3.1184
(128, 1500)
Epoch: 1/20... Step: 50... Loss: 3.1398... Val Loss: 3.1166
(128, 1500)
Epoch: 1/20... Step: 60... Loss: 3.1181... Val Loss: 3.1134
(128, 1500)
Epoch: 1/20... Step: 70... Loss: 3.1041... Val Loss: 3.1080
(128, 1500)
Epoch: 1/20... Step: 80... Loss: 3.1125... Val Loss: 3.0995
(128, 1500)
Epoch: 1/20... Step: 90... Loss: 3.0921... Val Loss: 3.0753
(128, 1500)
Epoch: 1/20... Step: 100... Loss: 3.0326... Val Loss: 3.0167
(128, 1500)
Epoch: 1/20... Step: 110... Loss: 2.9656... Val Loss: 2.9455
(128, 1500)
Epoch: 1/20... Step: 120... Loss: 2.8341... Val Loss: 2.8203
(128, 1500)
Epoch: 1/20... Step: 130... Loss: 2.7855... Val Loss: 2.8155
(128, 13900)
(128, 1500)
Epoch: 2/20... Step: 1

KeyboardInterrupt: ignored